# lil_cluster
Фундаментальная модель в ансамбле

Изучив специфику авторынка стало понятно, что бренд и модель - названия, не имеюще общих правил. Поэтому модель их может использовать только в качетсве dummy переменных. Но 500 с лишним переменных для меня показались слишком большим количеством для 80 тысяч автомобилей. Поэтому я решил разделить их специфику по моделям. Тут становится интереснее. Оказывается автомобили одной и той же модели, но разных комплектаций могут кратно отличаться в цене. Исходя из этого инсайта я понял, что можно сравнивать автомобили только в рамках одной модели. Этим и будет заниматься lil_cluster

Оказалось, что в датасете множество машин одной и той же модели, более того, в тренировочном датасете оказалось 98% наименований моделей, покрывающие наименования тестового датасета. Исходя из этого, lil_cluster будет иметь главный решающий голос в ансамбле в случаях, когда модель машины ему хорошо известна

In [1]:
%store -r df

In [2]:
df.columns

Index(['bodyType', 'brand', 'color', 'description', 'engineDisplacement',
       'enginePower', 'fuelType', 'mileage', 'model_name', 'numberOfDoors',
       'sell_id', 'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'price', 'model_product_time', 'car_age'],
      dtype='object')

Для этой модели потребуется сократить разновидности моделей в тренировочной выборки до соответствия их тестовой

In [3]:
import pandas as pd

In [4]:
test = df[df.price==-1]
train = df[df.price!=-1]

In [5]:
print(f'''{test.model_name.nunique()} - количество уникальных моделей 
{len(test.groupby(['brand','model_name']).count())} - количество\
уникальных моделей с привязкой к бренду

названия моделей в тестовом датасете - \
уникальны для всех предоставленных брендов''')

544 - количество уникальных моделей 
544 - количествоуникальных моделей с привязкой к бренду

названия моделей в тестовом датасете - уникальны для всех предоставленных брендов


In [6]:
print(f'''{train.model_name.nunique()} - количество уникальных моделей 
{len(train.groupby(['brand','model_name']).count())} - количество\
уникальных моделей с привязкой к бренду

20 моделей из тренировочного датасета из разных брендов имеют одинаковое название''')

1086 - количество уникальных моделей 
1105 - количествоуникальных моделей с привязкой к бренду

20 моделей из тренировочного датасета из разных брендов имеют одинаковое название


In [7]:
test = df[df.price==-1]
train = df[df.price!=-1]

### Исключает модели, которых нет в  ###
filtered_train = train[train.model_name.isin(test.model_name.unique())
                      ].reset_index(drop=True)



'''Делает неуникальные для разных брендов названия моделей - уникальными'''
###############################################################################
grouped_ft = filtered_train.groupby(['brand','model_name']).count()

valco_non_unique = grouped_ft.reset_index(level=1).model_name.value_counts()[
    grouped_ft.reset_index(level=1).model_name.value_counts()!=1]

def change_name_to_unique(df):
    df_non_unique = df[
        df.model_name.isin(valco_non_unique.index)]

    for i in df_non_unique.index:
        df.at[i,'model_name'] = \
        df.iloc[i].brand + '_' + df.iloc[i].model_name

change_name_to_unique(filtered_train)
change_name_to_unique(test)

### Удаляет модели в трейне, которых нет в тесте после уточнения модели брендом
models_to_drop = (set(filtered_train.model_name.unique()) -set(test.model_name.unique()))
index_to_drop = filtered_train[filtered_train.model_name.isin(models_to_drop)].index

filtered_train.drop(index_to_drop,inplace=True)
###############################################################################


In [8]:
unknown = set(test.model_name.unique())- set(filtered_train.model_name.unique())
display(unknown)
print('18 Моделей из тестового датасета нет в тренировочном')

{'300ZX',
 'ARTEON',
 'COROLLA_II',
 'GOLF_R32',
 'IST',
 'LANCER_RALLIART',
 'LM',
 'OTTI',
 'POLO_GTI',
 'PORTE',
 'PRESEA',
 'RSQ3',
 'SPARKY',
 'TORNEO',
 'WILL_CYPHA',
 'X4_M',
 'Z',
 'ZEST'}

18 Моделей из тестового датасета нет в тренировочном


Модель будет специфичной под тренировочный датасет. Далее я разделяю тестовый датасет, но, важно, чтобы при разделении и соединении датасетов не потерялся порядок, в котором идут авто. 

## Предобработка

In [9]:
for_lil_prediction   = test[~test.model_name.isin(unknown)].reset_index(drop=True)
for_other_prediction = test[test.model_name.isin(unknown)].reset_index(drop=True)

In [10]:
test.fuelType.value_counts()

бензин     28601
дизель      5800
гибрид       223
электро       55
газ            7
Name: fuelType, dtype: int64

In [11]:
filtered_train.fuelType.value_counts()

бензин     38032
дизель      9093
гибрид       265
электро       92
газ           10
Name: fuelType, dtype: int64

In [12]:
filtered_train[filtered_train.model_name == '3ER']

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,model_name,numberOfDoors,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,price,model_product_time,car_age
4176,купе,BMW,чёрный,Автомобиль в собственности с июля 2018. За 2 с...,3.0,306,бензин,204000,3ER,2.0,-1,AUTOMATIC,3.0,1,ORIGINAL,задний,LEFT,655000.0,2.0,13.8
4177,седан,BMW,белый,"Дoбрый день. Продаю BMW Срочно ,c честным прoб...",2.0,184,бензин,114000,3ER,4.0,-1,AUTOMATIC,3.0,0,ORIGINAL,задний,LEFT,1200000.0,0.0,5.8
4178,седан,BMW,синий,"Автомобиль в отличном состоянии, сел и поехал!...",2.2,170,бензин,211000,3ER,4.0,-1,AUTOMATIC,3.0,1,DUPLICATE,задний,LEFT,380000.0,2.0,17.8
4179,седан,BMW,серый,Цена на автомобиль указана с учетом скидки от ...,2.0,156,бензин,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,задний,LEFT,2530000.0,2.0,0.8
4180,седан,BMW,чёрный,"Всем привет, в связи с расширением жилой площа...",2.0,150,дизель,26209,3ER,4.0,-1,AUTOMATIC,1.0,0,ORIGINAL,задний,LEFT,2450000.0,1.0,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5521,седан,BMW,белый,Только 19 и 20 августа дни онлайн-продаж в Бор...,2.0,190,дизель,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,полный,LEFT,3812300.0,2.0,0.8
5522,седан,BMW,синий,Только 19 и 20 августа дни онлайн-продаж в Бор...,2.0,190,дизель,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,полный,LEFT,3693500.0,2.0,0.8
5523,седан,BMW,чёрный,Мы открыты! Купите новый BMW быстро и абсолютн...,2.0,184,бензин,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,полный,LEFT,3333000.0,2.0,0.8
5524,седан,BMW,синий,"Цена указана с учетом скидки при трейд-ин, кре...",2.0,190,дизель,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,полный,LEFT,3688500.0,2.0,0.8


Можно попробовать использовать DBSCAN и k-nearest neigbors


F k-nearest будет зависить от количества машин в моделях. до 7 будет использоваться 1 сосед после  - два, после 30 - 3, а после 200 - 4 и это максимум

In [15]:
popa = filtered_train[filtered_train.model_name == '3ER']
popa.head()

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,model_name,numberOfDoors,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,price,model_product_time,car_age
4176,купе,BMW,чёрный,Автомобиль в собственности с июля 2018. За 2 с...,3.0,306,бензин,204000,3ER,2.0,-1,AUTOMATIC,3.0,1,ORIGINAL,задний,LEFT,655000.0,2.0,13.8
4177,седан,BMW,белый,"Дoбрый день. Продаю BMW Срочно ,c честным прoб...",2.0,184,бензин,114000,3ER,4.0,-1,AUTOMATIC,3.0,0,ORIGINAL,задний,LEFT,1200000.0,0.0,5.8
4178,седан,BMW,синий,"Автомобиль в отличном состоянии, сел и поехал!...",2.2,170,бензин,211000,3ER,4.0,-1,AUTOMATIC,3.0,1,DUPLICATE,задний,LEFT,380000.0,2.0,17.8
4179,седан,BMW,серый,Цена на автомобиль указана с учетом скидки от ...,2.0,156,бензин,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,задний,LEFT,2530000.0,2.0,0.8
4180,седан,BMW,чёрный,"Всем привет, в связи с расширением жилой площа...",2.0,150,дизель,26209,3ER,4.0,-1,AUTOMATIC,1.0,0,ORIGINAL,задний,LEFT,2450000.0,1.0,1.8


In [16]:
filtered_train[filtered_train.model_name == '3ER']

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,model_name,numberOfDoors,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,price,model_product_time,car_age
4176,купе,BMW,чёрный,Автомобиль в собственности с июля 2018. За 2 с...,3.0,306,бензин,204000,3ER,2.0,-1,AUTOMATIC,3.0,1,ORIGINAL,задний,LEFT,655000.0,2.0,13.8
4177,седан,BMW,белый,"Дoбрый день. Продаю BMW Срочно ,c честным прoб...",2.0,184,бензин,114000,3ER,4.0,-1,AUTOMATIC,3.0,0,ORIGINAL,задний,LEFT,1200000.0,0.0,5.8
4178,седан,BMW,синий,"Автомобиль в отличном состоянии, сел и поехал!...",2.2,170,бензин,211000,3ER,4.0,-1,AUTOMATIC,3.0,1,DUPLICATE,задний,LEFT,380000.0,2.0,17.8
4179,седан,BMW,серый,Цена на автомобиль указана с учетом скидки от ...,2.0,156,бензин,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,задний,LEFT,2530000.0,2.0,0.8
4180,седан,BMW,чёрный,"Всем привет, в связи с расширением жилой площа...",2.0,150,дизель,26209,3ER,4.0,-1,AUTOMATIC,1.0,0,ORIGINAL,задний,LEFT,2450000.0,1.0,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5521,седан,BMW,белый,Только 19 и 20 августа дни онлайн-продаж в Бор...,2.0,190,дизель,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,полный,LEFT,3812300.0,2.0,0.8
5522,седан,BMW,синий,Только 19 и 20 августа дни онлайн-продаж в Бор...,2.0,190,дизель,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,полный,LEFT,3693500.0,2.0,0.8
5523,седан,BMW,чёрный,Мы открыты! Купите новый BMW быстро и абсолютн...,2.0,184,бензин,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,полный,LEFT,3333000.0,2.0,0.8
5524,седан,BMW,синий,"Цена указана с учетом скидки при трейд-ин, кре...",2.0,190,дизель,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,полный,LEFT,3688500.0,2.0,0.8


In [17]:
# Машина
'bodyType', 'engineDisplacement', 'enginePower', 'fuelType', 'vehicleTransmission', 'Привод'

('bodyType', 'engineDisplacement', 'enginePower', 'fuelType')

In [ ]:
# Важное не машина
'car_age', 'mileage'

In [ ]:
# человеческий фактор и маловажное
'color', 'description', 'Владельцы', 'Владение', 'ПТС', 'mileage', 'model_product_time', 'Руль', 'numberOfDoors'

In [18]:
df.columns

Index(['bodyType', 'brand', 'color', 'description', 'engineDisplacement',
       'enginePower', 'fuelType', 'mileage', 'model_name', 'numberOfDoors',
       'sell_id', 'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'price', 'model_product_time', 'car_age'],
      dtype='object')

# Машина

In [28]:
filtered_train[(filtered_train.model_name == '3ER') & (filtered_train.engineDisplacement == 2.0)]

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,model_name,numberOfDoors,sell_id,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,price,model_product_time,car_age
4177,седан,BMW,белый,"Дoбрый день. Продаю BMW Срочно ,c честным прoб...",2.0,184,бензин,114000,3ER,4.0,-1,AUTOMATIC,3.0,0,ORIGINAL,задний,LEFT,1200000.0,0.0,5.8
4179,седан,BMW,серый,Цена на автомобиль указана с учетом скидки от ...,2.0,156,бензин,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,задний,LEFT,2530000.0,2.0,0.8
4180,седан,BMW,чёрный,"Всем привет, в связи с расширением жилой площа...",2.0,150,дизель,26209,3ER,4.0,-1,AUTOMATIC,1.0,0,ORIGINAL,задний,LEFT,2450000.0,1.0,1.8
4181,седан,BMW,красный,Автомобиль проверен и готов к эксплуатации\nГа...,2.0,150,бензин,390983,3ER,4.0,-1,MECHANICAL,3.0,0,ORIGINAL,задний,LEFT,100000.0,2.0,28.8
4183,седан,BMW,серый,"Недостатки по лкп, мотор 2 литра N42B20",2.0,143,бензин,280000,3ER,4.0,-1,MECHANICAL,3.0,0,ORIGINAL,задний,LEFT,295000.0,3.0,16.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5521,седан,BMW,белый,Только 19 и 20 августа дни онлайн-продаж в Бор...,2.0,190,дизель,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,полный,LEFT,3812300.0,2.0,0.8
5522,седан,BMW,синий,Только 19 и 20 августа дни онлайн-продаж в Бор...,2.0,190,дизель,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,полный,LEFT,3693500.0,2.0,0.8
5523,седан,BMW,чёрный,Мы открыты! Купите новый BMW быстро и абсолютн...,2.0,184,бензин,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,полный,LEFT,3333000.0,2.0,0.8
5524,седан,BMW,синий,"Цена указана с учетом скидки при трейд-ин, кре...",2.0,190,дизель,0,3ER,4.0,-1,AUTOMATIC,0.0,0,ORIGINAL,полный,LEFT,3688500.0,2.0,0.8


In [27]:
filtered_train[filtered_train.model_name == '3ER'].engineDisplacement.value_counts()

2.0    989
2.5    125
3.0     61
1.6     52
1.8     36
2.2     32
1.9     21
1.5     20
2.8     12
2.9      1
1.7      1
Name: engineDisplacement, dtype: int64

In [33]:


pd.concat(
[
    
filtered_train[['engineDisplacement', 'enginePower']] ,
pd.get_dummies(filtered_train.bodyType),
pd.get_dummies(filtered_train.fuelType),
pd.get_dummies(filtered_train.vehicleTransmission),
pd.get_dummies(filtered_train.Привод)],axis=1)

,engineDisplacement,enginePower,внедорожник,компактвэн,купе,лифтбек,минивэн,пикап,седан,универсал,...,гибрид,дизель,электро,AUTOMATIC,MECHANICAL,ROBOT,VARIATOR,задний,передний,полный
0,2.8,174,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
1,1.8,90,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
2,2.3,136,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
3,1.8,90,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
4,2.0,101,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48020,3.0,150,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
48021,3.0,150,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
48022,3.0,258,0,0,0,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,1
48023,2.3,190,0,0,0,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,1


In [39]:
filtered_train[filtered_train.mileage==0].mileage.value_counts()

0    9666
Name: mileage, dtype: int64

In [42]:
test.mileage.mean()

162009.76788906186

In [41]:
test.mileage.median()

149779.5

In [ ]:
filtered_train.engineDisplacement

In [ ]:
'engineDisplacement' 'enginePower' 

In [ ]:
pd.get_dummies(filtered_train.bodyType)
pd.get_dummies(filtered_train.fuelType)
pd.get_dummies(filtered_train.vehicleTransmission)
pd.get_dummies(filtered_train.Привод)


pd.get_dummies(.Руль)
pd.get_dummies()
pd.get_dummies()

,бензин,газ,гибрид,дизель,электро
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
48020,1,0,0,0,0
48021,1,0,0,0,0
48022,0,0,0,1,0
48023,0,0,0,1,0


In [25]:
pd.concat([pd.get_dummies(filtered_train.bodyType) ,filtered_train],axis=1).corr()

,внедорожник,компактвэн,купе,лифтбек,минивэн,пикап,седан,универсал,хэтчбек,экстра,engineDisplacement,enginePower,mileage,numberOfDoors,sell_id,Владельцы,Владение,price,model_product_time,car_age
внедорожник,1.000000,-0.095662,-0.129297,-0.204452,-0.136097,-0.073090,-0.629896,-0.144927,-0.206257,-0.073881,0.274987,0.256620,-0.266019,0.610911,NaN,-0.289953,-0.078972,0.227985,-0.027512,-0.293006
компактвэн,-0.095662,1.000000,-0.018229,-0.028825,-0.019188,-0.010305,-0.088807,-0.020433,-0.029079,-0.010416,-0.086717,-0.100668,0.041197,0.025025,NaN,0.009218,0.008901,-0.050649,0.030271,0.041090
купе,-0.129297,-0.018229,1.000000,-0.038960,-0.025934,-0.013928,-0.120032,-0.027617,-0.039304,-0.014079,0.077090,0.131102,-0.023917,-0.575404,NaN,0.049180,0.020975,0.052905,-0.001936,0.052320
лифтбек,-0.204452,-0.028825,-0.038960,1.000000,-0.041009,-0.022024,-0.189801,-0.043670,-0.062150,-0.022262,-0.162766,-0.134981,-0.040440,0.188852,NaN,-0.030401,0.012469,-0.075791,0.014497,-0.095559
минивэн,-0.136097,-0.019188,-0.025934,-0.041009,1.000000,-0.014660,-0.126344,-0.029069,-0.041371,-0.014819,-0.014414,-0.068253,0.078584,-0.006277,NaN,0.021626,0.034382,0.019794,0.082388,0.040489
пикап,-0.073090,-0.010305,-0.013928,-0.022024,-0.014660,1.000000,-0.067853,-0.015612,-0.022218,-0.007958,0.034438,-0.011174,0.009006,-0.068293,NaN,-0.014056,0.013024,-0.001103,0.023124,-0.013239
седан,-0.629896,-0.088807,-0.120032,-0.189801,-0.126344,-0.067853,1.000000,-0.134542,-0.191477,-0.068586,-0.076284,-0.057930,0.163092,-0.546991,NaN,0.181836,0.011916,-0.113643,-0.038176,0.177408
универсал,-0.144927,-0.020433,-0.027617,-0.043670,-0.029069,-0.015612,-0.134542,1.000000,-0.044055,-0.015780,-0.061549,-0.068281,0.156905,0.134058,NaN,0.099021,0.037004,-0.083747,0.020137,0.146834
хэтчбек,-0.206257,-0.029079,-0.039304,-0.062150,-0.041371,-0.022218,-0.191477,-0.044055,1.000000,-0.022458,-0.232921,-0.221457,0.071124,0.048608,NaN,0.125980,0.044222,-0.129186,0.009313,0.127509
экстра,-0.073881,-0.010416,-0.014079,-0.022262,-0.014819,-0.007958,-0.068586,-0.015780,-0.022458,1.000000,0.030514,0.009246,0.036871,-0.247755,NaN,0.047715,0.027489,0.000206,0.049941,0.082661


,внедорожник,компактвэн,купе,лифтбек,минивэн,пикап,седан,универсал,хэтчбек,экстра
0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
48020,0,0,1,0,0,0,0,0,0,0
48021,0,0,1,0,0,0,0,0,0,0
48022,0,0,0,0,0,1,0,0,0,0
48023,0,0,0,0,0,1,0,0,0,0


In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

knn = KNeighborsClassifier(n_neighbors=3)

In [22]:
engineDisplacement

NameError: name 'engineDisplacement' is not defined